We will use the [`aif360`](https://aif360.readthedocs.io/en/latest/Getting%20Started.html) package to load the UCI adult dataset, fit a simple model and then analyse the fairness of the model using the DA-AUC.

In [3]:
from aif360.datasets import AdultDataset
from aif360.algorithms.preprocessing import Reweighing

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

The below code downloads the required files from the UCI website if they are not already present in ai360's data directory.

In [9]:
import urllib.request
import aif360
import os

aif360_path = os.path.dirname(aif360.__file__)
adult_path = os.path.join(aif360_path, "data", "raw", "adult")

# Define the file paths
files = ["adult.data", "adult.test", "adult.names"]
urls = [
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names",
]

# Check and download files if they do not exist
for file, url in zip(files, urls):
    file_path = os.path.join(adult_path, file)
    if not os.path.exists(file_path):
        print(f"{file} not found. Downloading from {url}...")
        urllib.request.urlretrieve(url, file_path)
        print(f"{file} downloaded.")
    else:
        print(f"{file} already exists.")

adult.data already exists.
adult.test already exists.
adult.names already exists.


In [10]:
privileged_groups1 = [{"sex": 1}]
unprivileged_groups1 = [{"sex": 0}]
privileged_groups2 = [{"race": 1}]
unprivileged_groups2 = [{"race": 0}]


dataset = AdultDataset()

(dataset_orig_train, dataset_orig_val) = dataset.split([0.7], shuffle=True)

In [11]:
model = make_pipeline(StandardScaler(), LogisticRegression(solver="liblinear", random_state=1))
fit_params = {"logisticregression__sample_weight": dataset_orig_train.instance_weights}

lr_orig = model.fit(dataset_orig_train.features, dataset_orig_train.labels.ravel(), **fit_params)

In [12]:
rw_sex = Reweighing(unprivileged_groups=unprivileged_groups1, privileged_groups=privileged_groups1)
rw_race = Reweighing(unprivileged_groups=unprivileged_groups2, privileged_groups=privileged_groups2)

trans_sex_dataset = rw_sex.fit(dataset).transform(dataset)
trans_sex_race_dataset = rw_race.fit(trans_sex_dataset).transform(trans_sex_dataset)

trans_sex_dataset_train = rw_sex.fit(dataset_orig_train).transform(dataset_orig_train)
trans_sex_race_dataset_train = rw_race.fit(trans_sex_dataset_train).transform(trans_sex_dataset_train)

model = make_pipeline(StandardScaler(), LogisticRegression(solver="liblinear", random_state=1))
fit_params = {"logisticregression__sample_weight": trans_sex_race_dataset_train.instance_weights}
lr_rw = model.fit(trans_sex_race_dataset_train.features, trans_sex_race_dataset_train.labels.ravel(), **fit_params)

In [ ]:
from daindex.core import deterioration_index
import numpy as np

def obtain_det_alo_index(
    df,
    scores,
    det_feature,
    score_threshold,
    cohort_name,
    det_label,
    score_margin=0.05,
    det_threshold=3,
    min_v=-5,
    max_v=15,
    reverse=False,
    is_discrete=True,
    search_bandwidth=True,
    det_feature_func=None,
):
    lb = score_threshold - score_margin
    up = score_threshold + score_margin
    det_list = []
    for i, r in df.iterrows():
        p = scores[i]
        if lb <= p <= up:
            if det_feature_func is not None:
                det_list.append(det_feature_func(r))
            else:
                det_list.append(r[det_feature])
    if len(det_list) > 20:
        X = np.array(det_list)
        di_ret = deterioration_index(
            X[~np.isnan(X)].reshape(-1, 1),
            min_v,
            max_v,
            threshold=det_threshold,
            plot_title=f"{cohort_name} | {det_label}",
            reverse=reverse,
            is_discrete=is_discrete,
            search_bandwidth=search_bandwidth,
            do_plot=False,
        )
        return score_threshold, len(det_list), di_ret["k-step"]
    else:
        return score_threshold, 0, 0


def get_scores(models, df, features):
    predicted_probs = np.array([m.predict_proba(df[features].to_numpy()) for m in models])
    return predicted_probs[:, :, 1].mean(axis=0)


df_white = dataset[dataset["ethnicity"] == 1]
df_non_white = dataset[dataset["ethnicity"] == 0]

df_male = kidney_df[kidney_df["gender"] == 1]
df_female = kidney_df[kidney_df["gender"] == 0]

steps = 50
det_feature = "Creatinine Max"
det_label = "Creatinine Max"
is_discrete = False
min_v = min(np.min(df_white[det_feature]), np.min(df_non_white[det_feature]))
max_v = max(np.max(df_white[det_feature]), np.max(df_non_white[det_feature]))
print(min_v, max_v)
det_threshold = 1.35  # .7

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, precision_recall_curve, auc
from aif360.metrics import ClassificationMetric
from collections import defaultdict
from metrics.eval_metrics import print_metrics_binary


def compute_deterioration_index(clf):
    white_ret = []
    non_white_ret = []
    male_ret = []
    female_ret = []

    scores1 = get_scores(clf, df_white, feature_list)
    scores2 = get_scores(clf, df_non_white, feature_list)
    scores3 = get_scores(clf, df_male, feature_list)
    scores4 = get_scores(clf, df_female, feature_list)

    # Calculate DA AUC for this fold and accumulate the values
    for s in range(1, steps + 1):
        white_ret.append(
            obtain_det_alo_index(
                df_white,
                scores1,
                det_feature,
                s / steps,
                cohort_name="White cohort",
                det_label=det_label,
                det_threshold=det_threshold,
                min_v=min_v,
                max_v=max_v,
                is_discrete=is_discrete,
            )
        )
        non_white_ret.append(
            obtain_det_alo_index(
                df_non_white,
                scores2,
                det_feature,
                s / steps,
                cohort_name="Non-White cohort",
                det_label=det_label,
                det_threshold=det_threshold,
                min_v=min_v,
                max_v=max_v,
                is_discrete=is_discrete,
            )
        )
        male_ret.append(
            obtain_det_alo_index(
                df_male,
                scores3,
                det_feature,
                s / steps,
                cohort_name="Male cohort",
                det_label=det_label,
                det_threshold=det_threshold,
                min_v=min_v,
                max_v=max_v,
                is_discrete=is_discrete,
            )
        )
        female_ret.append(
            obtain_det_alo_index(
                df_female,
                scores4,
                det_feature,
                s / steps,
                cohort_name="Female cohort",
                det_label=det_label,
                det_threshold=det_threshold,
                min_v=min_v,
                max_v=max_v,
                is_discrete=is_discrete,
            )
        )

    return white_ret, non_white_ret, male_ret, female_ret

In [ ]:
def calculate_daauc_values_and_graphs(results):
    # Visualization of Average DA AUC Values for Each Model
    viz_config = {"fig_size": (4, 3), "font_size": 12}

    # Loop through each model in all_results to generate the visualizations
    for model_name, _ in models.items():
        print(f"\n\n{model_name}")
        for i in range(0, 10):
            white_ret = results[model_name][f"fold_{i}"]["white_ret"]
            non_white_ret = results[model_name][f"fold_{i}"]["non_white_ret"]
            male_ret = results[model_name][f"fold_{i}"]["male_ret"]
            female_ret = results[model_name][f"fold_{i}"]["female_ret"]

            print("\n===============")
            print(i)
            print("ethnicity:")
            # Visualize White vs Non-White average DA AUC
            a1, a2, da1, da2 = viz(
                np.array(white_ret),
                np.array(non_white_ret),
                "White",
                "Non-White",
                f"{det_label} {'>='}{det_threshold}",
                f"{model_name}",
                viz_config,
            )
            results[model_name][f"fold_{i}"]["ethnicity_auc"] = (a2 - a1) / a1
            try:
                results[model_name][f"fold_{i}"]["ethnicity_dauc"] = (da2 - da1) / da1
            except ZeroDivisionError:
                results[model_name][f"fold_{i}"]["ethnicity_dauc"] = 0
            print("sex:")
            # Visualize Male vs Female average DA AUC
            a1, a2, da1, da2 = viz(
                np.array(male_ret),
                np.array(female_ret),
                "Male",
                "Female",
                f"{det_label} {'>='}{det_threshold}",
                f"{model_name}",
                viz_config,
            )
            results[model_name][f"fold_{i}"]["sex_auc"] = (a2 - a1) / a1
            try:
                results[model_name][f"fold_{i}"]["sex_dauc"] = (da2 - da1) / da1
            except ZeroDivisionError:
                results[model_name][f"fold_{i}"]["sex_dauc"] = 0
    return results